In [1]:
from humemai.janusgraph import Humemai

humemai = Humemai("./humemai/janusgraph/docker-compose-cql-es.yml")
humemai.connect()


DEBUG:humemai.janusgraph.utils.docker:Docker Compose started successfully.
DEBUG:humemai.janusgraph.utils.docker:
INFO:gremlinpython:Creating DriverRemoteConnection with url 'ws://localhost:8182/gremlin'
INFO:gremlinpython:Creating Client with url 'ws://localhost:8182/gremlin'
INFO:gremlinpython:Creating GraphTraversalSource.
INFO:gremlinpython:Creating GraphTraversalSource.
DEBUG:humemai.janusgraph.humemai:Successfully connected to the Gremlin server.


In [2]:
humemai.remove_all_data()

DEBUG:gremlinpython:submit with bytecode '[['V'], ['drop'], ['none']]'
DEBUG:gremlinpython:message '[['V'], ['drop'], ['none']]'
DEBUG:gremlinpython:processor='traversal', op='bytecode', args='{'gremlin': [['V'], ['drop'], ['none']], 'aliases': {'g': 'g'}}'
DEBUG:asyncio:Using selector: EpollSelector


In [3]:
humemai.g.V().toList()

DEBUG:gremlinpython:submit with bytecode '[['V']]'
DEBUG:gremlinpython:message '[['V']]'
DEBUG:gremlinpython:processor='traversal', op='bytecode', args='{'gremlin': [['V']], 'aliases': {'g': 'g'}}'
DEBUG:asyncio:Using selector: EpollSelector


[]

In [4]:
humemai.disconnect()

INFO:gremlinpython:closing DriverRemoteConnection with url 'ws://localhost:8182/gremlin'
INFO:gremlinpython:Closing Client with url 'ws://localhost:8182/gremlin'
DEBUG:humemai.janusgraph.humemai:Disconnected from the Gremlin server.


In [5]:
humemai.stop_docker_compose()

DEBUG:humemai.janusgraph.utils.docker:Docker Compose stopped successfully.
DEBUG:humemai.janusgraph.utils.docker:


In [13]:
humemai.remove_docker_compose()

DEBUG:humemai.janusgraph.utils.docker:Docker Compose removed successfully.
DEBUG:humemai.janusgraph.utils.docker:
